In [17]:
import pickle
import pandas as pd
import json

import torch
from tqdm import tqdm

In [2]:
vocab_tokens = json.load(open("./dataset/recipe1m_vocab_tokens.json"))

In [3]:
det_df = pd.read_csv("./dataset/final_base_ingrs.csv")

In [4]:
det_df.head()

,id,ingredients,base_ingrs
0,000018c8a5,penne;cheese_sauce;cheddar_cheese;gruyere_chee...,salt;garlic;dried_chipotle_powder;cheese;penne...
1,000033e39b,elbow_macaroni;american_cheese;celery;green_pe...,celery;dill;salt;cheese;mayonnaise;pimentos;pe...
2,000035f7ed,tomatoes;kosher_salt;red_onion;green_bell_pepp...,tomato;salt;oil;cucumber;pepper;onion;basil
3,00003a70b1,milk;water;butter;mashed_potatoes;whole_kernel...,potato;water;cheese;corn;butter;milk;onion
4,00004320bb,watermelon_gelatin;boiling_water;cool_whip;see...,gelatin;cracker;water;watermelon;cool_whip


In [5]:
available_ids = list(det_df["id"])

In [18]:
def get_tokens(data):
    vocab_words = vocab_tokens.keys()
#     filter_data = filter(lambda x: x["id"] in available_ids, data)
#     filter_data = [x for x in data if x["id"] in available_ids]

    inst_tokens = []
    
    for inst in tqdm(data):
        if (inst["id"] in available_ids):
            ingrs = det_df[det_df["id"] == inst["id"]]["base_ingrs"].values[0]
            ingrs = " ".join(ingrs.split(";"))

            raw_title = " ".join(inst["title"])
            raw_inst = [" ".join(line) for line in inst["tokenized"]]
            raw_inst = " <next_inst> ".join(raw_inst)
            raw_inst = f"<start> {ingrs} <inst_start> {raw_inst} <inst_end> <title_start> {raw_title} <title_end> <end>"

            tokens = [vocab_tokens[word] for word in raw_inst.split(" ") if word in vocab_words]
            inst_tokens.append(tokens)
    
    return torch.tensor(inst_tokens)

In [19]:
test_data = pickle.load(open("./dataset/pickle/recipe1m_test.pkl", "rb"))
test_tokens = get_tokens(test_data)

torch.save(test_tokens, './dataset/test_tokens.pt')

  1%|▍                                                                        | 775/138070 [01:00<2:57:19, 12.90it/s]


KeyboardInterrupt: 

In [ ]:
train_data = pickle.load(open("./dataset/pickle/recipe1m_train.pkl", "rb"))
# train_tokens = get_tokens(train_data)

# torch.save(train_tokens, './dataset/train_tokens.pt')